<a href="https://colab.research.google.com/github/halim-jun/Ai_study/blob/master/202008_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

In [ ]:
files.upload()

In [76]:
import pandas as pd
train=pd.read_csv("train.csv")

In [77]:
test=pd.read_csv("test.csv")
test_2=pd.read_csv("gender_submission.csv")
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


#1. Data 전처리

##1) 테스트 데이터 전처리
- cabin, Name, ticket number, passenger number drop
- X, Y set 으로 분리

In [78]:
total_test=pd.merge(test, test_2, how='inner', left_on='PassengerId', right_on='PassengerId')
total_test.dropna(axis=0, how='any', inplace=True)
total_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
12,904,1,"Snyder, Mrs. John Pillsbury (Nelle Stevenson)",female,23.0,1,0,21228,82.2667,B45,S,1
14,906,1,"Chaffee, Mrs. Herbert Fuller (Carrie Constance...",female,47.0,1,0,W.E.P. 5734,61.1750,E31,S,1
24,916,1,"Ryerson, Mrs. Arthur Larned (Emily Maria Borie)",female,48.0,1,3,PC 17608,262.3750,B57 B59 B63 B66,C,1
26,918,1,"Ostby, Miss. Helene Ragnhild",female,22.0,0,1,113509,61.9792,B36,C,1
28,920,1,"Brady, Mr. John Bertram",male,41.0,0,0,113054,30.5000,A21,S,0


In [79]:
y_test=total_test['Survived']
y_test.head()

12    1
14    1
24    1
26    1
28    0
Name: Survived, dtype: int64

In [80]:
total_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
12,904,1,"Snyder, Mrs. John Pillsbury (Nelle Stevenson)",female,23.0,1,0,21228,82.2667,B45,S,1
14,906,1,"Chaffee, Mrs. Herbert Fuller (Carrie Constance...",female,47.0,1,0,W.E.P. 5734,61.1750,E31,S,1
24,916,1,"Ryerson, Mrs. Arthur Larned (Emily Maria Borie)",female,48.0,1,3,PC 17608,262.3750,B57 B59 B63 B66,C,1
26,918,1,"Ostby, Miss. Helene Ragnhild",female,22.0,0,1,113509,61.9792,B36,C,1
28,920,1,"Brady, Mr. John Bertram",male,41.0,0,0,113054,30.5000,A21,S,0


In [81]:
x_test=total_test.drop(labels=['Cabin','Name','Ticket', 'PassengerId'], inplace=False, axis=1)

In [82]:
x_test.drop(labels=['Survived'], inplace=True, axis=1)

In [83]:
x_test.replace(to_replace='S', value=0, inplace=True)
x_test.replace(to_replace='C', value=1, inplace=True)
x_test.replace(to_replace='Q', value=2, inplace=True)
x_test.replace(to_replace='male', value=1, inplace=True)
x_test.replace(to_replace='female', value=0, inplace=True)

In [84]:
x_test.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

##2. Train data 전처리 
1) 기본 
- null, na 제거
- 불필요한 칼럼 제거

In [85]:
import sklearn

In [86]:
train.isna().sum()
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [87]:
train.drop(labels=['Cabin','Name', 'PassengerId','Ticket'], axis=1, inplace=True)

In [88]:
train_withage=train.dropna(axis=0, how='any', inplace=False)
train_withage.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

## 2) Accuracy Paradox 확인

In [89]:
train_withage['Survived'].describe()

count    712.000000
mean       0.404494
std        0.491139
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Survived, dtype: float64

## 3) Dummy Variable 처리

In [90]:
train_withage['Embarked'].unique()
train_withage.replace(to_replace='male', value=1, inplace=True)
train_withage.replace(to_replace='female', value=0, inplace=True)
train_withage.replace(to_replace='S', value=0, inplace=True)
train_withage.replace(to_replace='C', value=1, inplace=True)
train_withage.replace(to_replace='Q', value=2, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


## 4) Label, Feature 분리

In [91]:
from sklearn.model_selection import train_test_split
label=train_withage['Survived']
feature=train_withage.drop(labels='Survived', axis=1, inplace=False)

In [92]:
feature.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

#2. Model selection
- Best accuracy 인 classifier model 로 선정

In [93]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

In [94]:
model_tree=DecisionTreeClassifier(random_state=32)
model_tree.fit(feature, label)
y_pred=model_tree.predict(x_test)
tree_acc=accuracy_score(y_pred, y_test)

In [95]:
tree_acc

0.8275862068965517

In [96]:
model_svm=SVC()
model_svm.fit(feature, label)
y_pred=model_svm.predict(x_test)
svm_acc=accuracy_score(y_test, y_pred)

In [97]:
model_rand=RandomForestClassifier()
model_rand.fit(feature, label)
y_pred=model_rand.predict(x_test)
rand_acc=accuracy_score(y_test, y_pred)

In [98]:
model_Logi=LogisticRegression()
model_Logi.fit(feature, label)
y_pred=model_Logi.predict(x_test)
Logi_acc=accuracy_score(y_test, y_pred)

In [99]:
model_SGD=SGDClassifier()
model_SGD.fit(feature, label)
y_pred=model_SGD.predict(x_test)
SGD_acc=accuracy_score(y_test, y_pred)

In [100]:
pd.DataFrame([['tree',tree_acc], ['svm',svm_acc], ['random',rand_acc], ['Logistic regressopn',Logi_acc], ['Stochastic Gradient Logisitc',SGD_acc]], columns=['Algorithm', 'Accuracy'])

,Algorithm,Accuracy
0,tree,0.827586
1,svm,0.620690
2,random,0.873563
3,Logistic regressopn,0.850575
4,Stochastic Gradient Logisitc,0.609195
